In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import cv2
import os

# Czytanie danych

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

train = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/train')
test = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/test')
val = get_training_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/val')

In [ ]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

# Przygotowywanie danych

In [ ]:
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

def to_rgb(x):
    x_rgb = np.zeros((x.shape[0], img_size, img_size, 3))
    for i in range(3):
        x_rgb[..., i] = x[..., 0]
    return x_rgb

x_train = to_rgb(x_train)
x_test = to_rgb(x_test)

# Tworzenie sieci

In [ ]:
model = keras.Sequential(
    [
     keras.Input(shape=(img_size,img_size,3)),
     layers.Conv2D(16, 3, activation='relu'),
     layers.MaxPooling2D(),
     layers.Flatten(),
     layers.Dense(10)
    ]
)

In [ ]:
model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=keras.optimizers.Adam(),
  metrics=['accuracy']
)

# Tranowanie

In [ ]:
model.fit(
        x_train, 
        y_train, 
        epochs=120, 
        batch_size=32, 
        validation_data = (x_test, y_test))

In [ ]:
print("Dokładność: " , model.evaluate(x_test,y_test)[1]*100 , "%")

#  Warstwy modelu

In [ ]:
model.summary() 

In [ ]:
predictions = model.predict(x_test)
predictions = predictions.reshape(1,-1)[0]
predictions=np.where(predictions > 0.5, 1,0)

# Funkcja heatmapy gran cam

In [ ]:
def gradcam(img, model, layer):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer).output, model.output])
    with tf.GradientTape() as tape:
        layer_output,grad_predictions=grad_model(img)
        class_channel=grad_predictions[:,tf.argmax(grad_predictions[0])]

    gradients=tape.gradient(class_channel,layer_output)
    gradients = tf.reduce_mean(gradients,axis=(0,1,2))

    heatmap=layer_output[0]@gradients[..., tf.newaxis]
    heatmap=tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap=heatmap.numpy()
    return heatmap

# Uruchomienie funkcji na całym zbiorze testowym

In [ ]:
model.layers[-1].activation = None

In [ ]:


for i,x in enumerate(x_test):
    if i!=0:
        img_array=keras.applications.xception.preprocess_input(x[np.newaxis,...])
        
        if predictions[i]==1:
            print("Predicted: Pneumonia")
        else:
            print("Predicted: Normal")
        
        heatmap = gradcam(img_array, model, "conv2d")
        
        plt.matshow(heatmap)
        plt.show()